# Generate exchange data using dedicated service - C#

### Overview
Reference data encompasses a wide range of specification data about:
* financial instrument such as asset class, symbol, maturity, etc.
* counterparties such as issuer information.
* pricing such as Open, high low and close prices.

The challenge with reference data is that it tends to be sourced from multiple sources: internal, counterparties and providers.

Reference data dedicated service is a result of various data sources and fields sourcing, validation, cross-checking and normalization.<br>

This sample requests **on-demand** exchange data and enables fields mapping by calling a dedicated static data service, making available clean data.

### Inputs/outputs
Exchange data sample is designed to help you searching and requesting exchange reference data by using a wide range of commonly used fields.

It takes exchange identifier as per input filter and returns the *entries* matching the *request*:
* List of instruments' identifiers
* Countries
* Currencies 
* Tick size rules
* etc. 

### Services used
This sample uses *gRPC requests* in order to retrieve exchange reference data from the hosted service. The queried endpoint in this script are:
* *StaticDataService*: to directly retrieve reference data objects from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.StaticData.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*

***

# Run Exchange data sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis"
#r "nuget: CsvHelper"

In [ ]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.StaticData.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
// Import/export csv files
using CsvHelper;
using System.IO;
using System.Globalization;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
var headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
var channel = ChannelHelpers.GetChannel();

The following code snippet enables to select the **exchange** by its *Market Identifier Code*:

In [ ]:
// Set exchange
var exchange = "XNGS";

The following code snippets call the service, generate the request and return the reply: 

In [ ]:
// Instantiate the service
var service = new StaticDataService.StaticDataServiceClient(channel);

// Generate request
var request = new StaticDataRequest { AssetType = AssetType.Equity, Exchange = exchange, Count = 100};

// Call the service
var reply = service.StaticData(request, headers);

// Display the results
//display(reply.Equities)

### Step 4: Retrieve data

#### 4.1 Retrieve instruments' identifiers (mapping)

In [ ]:
// Create a class to handle the instrument's mapping codes
public class Instrument
{
    public string Ticker{get; set;}
    public string Name{get; set;}
    public string Isin{get; set;}
    public string Cusip{get; set;}
    public string Sedol{get; set;}
    public string Bloomberg{get; set;}
}
var instruments = new List<Instrument>();

// Get mapping codes for the requested instruments
foreach (var reply in reply.Equities) 
{
    // Get mapping codes
    var ticker = reply.Identifier.Ticker;
    reply.Mapping.TryGetValue("Bloomberg", out var bloomberg);
    
    // Add the instruments
    instruments.Add(new Instrument(){Ticker = ticker, Name = reply.Name, Isin = reply.Isin, Cusip = reply.Cusip, Sedol = reply.Sedol, Bloomberg = bloomberg});
}

In [ ]:
display(instruments)

The following code snippet exports data to *csv* file:

In [ ]:
using (var writer = new StreamWriter($"{exchange}.csv"))
using (var csv = new CsvWriter(writer, CultureInfo.InvariantCulture))
{
    csv.WriteRecords(instruments);
}

#### 4.2 Retrieve countries

In [ ]:
var countries = reply.Equities.Select(r => r.Country).Distinct();
display(countries)

#### 4.3 Retrieve currencies

In [ ]:
var currencies = reply.Equities.Select(r => r.Currency).Distinct();
display(currencies)

#### 4.4 Retrieve tick size rules

In [ ]:
var tsr = reply.Equities.Select(r => r.TickSizeRule).Distinct();
display(tsr)